# Summary of basic SQL Queries

Up to 6 clauses in a query but only 2 mandatory, following the order:

```sql
SELECT      <attribute list>
FROM        <table list>
[WHERE      <condition>]
[GROUP BY   <grouping attribute(s)>]
[HAVING     <group condition>]
[ORDER BY   <attribute list>]
```

    SELECT:     lists the attributes/functions to be retrieved
    FROM:       specifies all realtions (or aliases) needed in the query but not those needed in nested queries
    WHERE:      specifies the conditions for selections of tuples from the relations specified in the FROM clause
    GROUP BY:   specifies grouping attributes
    HAVING:     specifies a condition for selection of groups
    ORDER BY:   specifies an order for displaying the results of the query

Query evaluation applies in the order:
1. WHERE 
2. GROUP-BY & HAVING
3. SELECT

# SQL Update Language

Moving on to insert/delete/change-ing data into/in database

- Insertions
- Updates
- Deletions

## Insertions

Insert tuples

    INSERT INTO R VALUES (v1,...,vk)

- values may be left NULL
- Can use results of queries for insertion

#### Examples

```sql
INSERT  INTO Movie
VALUES      ("Matchpoint", "Allen", "Allen")
```

```sql
INSERT  INTO Movie(Title, Director)
VALUES      ("Matchpoint", "Allen")
```

```sql
INSERT  INTO BertoMovie
SELECT  *    FROM Movie
WHERE   Director = "Berto"
```


## Deletions

Delete every tuple that satisfies <cond>

    DELETE FROM R WHERE <cond>

#### Example: Delete all movies that are not currently playing

```sql
DELETE  FROM    Movie
        WHERE   Title NOT IN (
                SELECT  Title
                FROM    Schedule
        )
```

> Nested queries in the WHERE clause provides a lot of expressive power for deletions

## Updates

Basic form: Update every tuple that satisfies <cond> in the way specified by the SET clause

    UPDATE  R
    SET     A1=<exp1>, ..., Ak=<expk>
    WHERE   <cond>

#### Examples

Change all "Berto" entries to "Bertolucci"

```sql
UPDATE  Movie
SET     Director="Bertolucci"
WHERE   Director="Berto"
```

Increase all salaries in the toys dept by 10%

```sql
UPDATE  Employee
SET     Salary=1.1*Salary
WHERE   Dept="Toys"
```

## More complex examples

Delete all theatres that show more than one title

```sql
DELETE  FROM schedule s
WHERE   EXISTS (
        SELECT  *
        FROM    schedule
        WHERE   theater =   s.theater
            AND title   <>  s.title
)
```

On the following data:

    Schedule    | Theater   Title
                | Hillcrest Amour
                | Hillcrest 0 dark 30
                | Paloma    Django

Standards group avoided a potential issue:

Issue: Say `Hillcrest Amour` gets deleted "first", then `Hillcrest 0 dark 30` wouldn't fulfill the <cond> and would stay. However, the system does not understand order, is non-deterministic in it's processing; this would lead to inconsistency.

Fix: Take a snapshot of the table before processing, and all of the <cond> would be evaluated using the snapshot of the table. Thus both `Hillcrest Amour` and `Hillcrest 0 dark 30` would get deleted.

Reminds me of spreadsheets recognizing circular logic or cell dependency issues and notifying u w/ an error message!


## Views, Assertions, & Triggers

**Views**: mechanism for customizing the database; also used for creating temporary virtual tables

**Assertions**: provide a mean to specify additional constraints

**Triggers**: are a special kind of assertions; they define actions to be taken when certain conditions occur

### Basic DBMS Architecture

![](images/2022-10-25-21-04-23.png)

### Views

**Use Case**: situatiosn where it is not desirable for all users to see the entire logical model (i.e. all relations stored in DB)

> consider a person who needs to know customers' loan numbers but has no need to see the loan ammounts. This person should see a relation described, in SQL, by:

```sql
SELECT	customer_name, loan_number
FROM    customer c, borrower b
WHERE	c.customer_id = b.customer_id
```

Mechanism for 
- Hiding
- Restructuring

Any relation that is not in the database schema but is made visible to the user as a "virtual relation" is called a **view**

#### View Definition

**Syntax**

```sql
CREATE VIEW V as <query expression>
```

- V: view name
    - opt. list of attrib. names
- <query expression>: any legal SQL query

**Notes**
- Once defined, view name can be used in queries just as any other relation
- Only limited updates can be applied to views (explained later)
- View Definition is NOT the same as creating a new relation by evaluating the query expression. view contents **changes automatically when the database is updated**

#### Example

View consisting of bank branches and all their cusomters

```sql
CREATE VIEW all_customers AS

(
    SELECT  branch_name, customer_id
    FROM    depositor d, account a
    WHERE   d.account_number = a.account_number
)

UNION

(
    SELECT  branch_name, customer_id
    FROM    borrower b, loan l
    WHERE   b.loan_number = l.loan_number
)
```

Example of using the view in a query:

Find all customers of the La Jolla branch

```sql
SELECT	customer_id
FROM	all_customers               -- virtual, system will lazily compute the filter & projection
WHERE	branch_name = 'La Jolla'
```

#### Views defined using other views

One view may be used in the expression defining another view

> Actually happens a lot in practice, often times the views don't even know they're using another view to evaluate expressions

- A view relation $V_1$ is said to **depend directly** on a view relation $V_2$ if $V_2$ is used in the expression defining $V_1$

- A iew relation $V_1$ is said to **depend on** view relation $V_2$ if either $V_1$ **depends directly** to $V_2$ or there is a path of dependencies from $V_1$ to $V_2$

- A view relation $V$ is said to be **recursive** if it depends on itself (will be discussed in depth later on...)



#### Views can simplify complex queries 

EX: Find actors playing in every movie by "Berto"

```sql
CREATE VIEW Berto-Movies AS
    SELECT	Actor
    FROM	Movie
    WHERE	Director = "Berto"
```

```sql
CREATE VIEW Not-All-Berto AS
    SELECT	m.Actor
    FROM	Movie m, Berto-Movies
    WHERE	Berto-Movies.Title NOT IN (
            SELECT  Title
            FROM    Movies
            WHERE   Actor = m.Actor
    )
```

```sql
SELECT  Actor
FROM    Movies
WHERE   Actor NOT IN (
        SELECT  *
        FROM    Not-All-Berto
)
```

#### Another sytnax: WITH clause

WITH: works syntactically almost identitcal to the CREATE VIEW statement

> Note: will only create temporary tables, not views

```sql
WITH Berto-Movies AS
    SELECT	Actor
    FROM	Movie
    WHERE	Director = "Berto"
```

```sql
WITH Not-All-Berto AS
    SELECT	m.Actor
    FROM	Movie m, Berto-Movies
    WHERE	Berto-Movies.Title NOT IN (
            SELECT  Title
            FROM    Movies
            WHERE   Actor = m.Actor
    )
```

```sql
SELECT  Actor
FROM    Movies
WHERE   Actor NOT IN (
        SELECT  *
        FROM    Not-All-Berto
)
```

> Note: will only create temporary tables, not views

#### Efficient view implementation

**Materialized views**

Physically create and maintain a view table

Assumption: other queries on the view will follow

Concerns: maintaining correspondance between the base table and the view when the base table is updated

Strategy: incremental updates

> note: will already be computed when called upon in other queries so will be much faster

**Virtual views**

Never physicaly created: Answer queries on the view by reformulating it as a query on the underlying base tables (by replacing the views by their definitions)

Disadvantage: Inefficient for views defined via complex queries (especially if additional queries are to be applied to the view within a short time period)

Advantage: No need to maintain correspondance with base tables

> This process is known as **view unfolding**

##### Query answering in the presence of virtual views

![](images/2022-10-25-21-32-57.png)

##### Examples of View Unfoldign:

![](images/2022-10-25-21-33-36.png)

![](images/2022-10-25-21-34-00.png)

![](images/2022-10-25-21-35-38.png)